## Extract: 
Found data sets from Kaggle.com on the World Happiness Report based off the Gallup World Poll. This includes two CSV files, one with happiness data from 2005-2020, and one with happiness data from 2021.  We extracted these CSV files and saved them in our repository's Resources folder.

In [1]:
# Import Dependecies
import pandas as pd
from sqlalchemy import create_engine, inspect

### Read CSV files into data frames

In [2]:
file1 = "Resources/world-happiness-report.csv"
multihappy_df = pd.read_csv(file1)
multihappy_df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [3]:
file2 = "Resources/world-happiness-report-2021.csv"
happy21_df = pd.read_csv(file2)
happy21_df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


## Transform: 
Extracted the columns we needed and renamed them.  We then added one column to each data frame necessary for the union.  We performed the union and created a lookup data frame to fill in missing column data.  Performed final merge and conducted final column cleanup for loading in to sql database.

In [4]:
# Create a filtered dataframe from specific columns
multihappy_cols = ["Country name", "year", "Life Ladder"]
multihappy_trans = multihappy_df[multihappy_cols].copy()

# Rename the column headers
multihappy_trans = multihappy_trans.rename(columns={"Country name": "Country Name",
                                                    "year": "Year",
                                                    "Life Ladder": "Ladder Score"})

multihappy_trans.head()

,Country Name,Year,Ladder Score
0,Afghanistan,2008,3.724
1,Afghanistan,2009,4.402
2,Afghanistan,2010,4.758
3,Afghanistan,2011,3.832
4,Afghanistan,2012,3.783


In [5]:
# Create a filtered dataframe from specific columns
happy21_cols = ["Country name", "Regional indicator", "Ladder score"]
happy21_trans = happy21_df[happy21_cols].copy()

# Rename the column headers
happy21_trans = happy21_trans.rename(columns={"Country name": "Country Name",
                                              "Regional indicator": "Regional Indicator",
                                              "Ladder score": "Ladder Score"})

happy21_trans.head()

,Country Name,Regional Indicator,Ladder Score
0,Finland,Western Europe,7.842
1,Denmark,Western Europe,7.620
2,Switzerland,Western Europe,7.571
3,Iceland,Western Europe,7.554
4,Netherlands,Western Europe,7.464


In [6]:
# Add column 
happy21_trans["Year"] = 2021
happy21_trans.head()

,Country Name,Regional Indicator,Ladder Score,Year
0,Finland,Western Europe,7.842,2021
1,Denmark,Western Europe,7.620,2021
2,Switzerland,Western Europe,7.571,2021
3,Iceland,Western Europe,7.554,2021
4,Netherlands,Western Europe,7.464,2021


In [7]:
# Add column
multihappy_trans["Regional Indicator"] = ""
multihappy_trans.head()

,Country Name,Year,Ladder Score,Regional Indicator
0,Afghanistan,2008,3.724,
1,Afghanistan,2009,4.402,
2,Afghanistan,2010,4.758,
3,Afghanistan,2011,3.832,
4,Afghanistan,2012,3.783,


In [8]:
frames = [multihappy_trans, happy21_trans]
allhappy = pd.concat(frames)
allhappy

,Country Name,Year,Ladder Score,Regional Indicator
0,Afghanistan,2008,3.724,
1,Afghanistan,2009,4.402,
2,Afghanistan,2010,4.758,
3,Afghanistan,2011,3.832,
4,Afghanistan,2012,3.783,
...,...,...,...,...
144,Lesotho,2021,3.512,Sub-Saharan Africa
145,Botswana,2021,3.467,Sub-Saharan Africa
146,Rwanda,2021,3.415,Sub-Saharan Africa
147,Zimbabwe,2021,3.145,Sub-Saharan Africa


In [9]:
# Create lookup data frame
ri_dfcols = ["Country Name", "Regional Indicator"]
ri_df = happy21_trans[ri_dfcols]
ri_df

,Country Name,Regional Indicator
0,Finland,Western Europe
1,Denmark,Western Europe
2,Switzerland,Western Europe
3,Iceland,Western Europe
4,Netherlands,Western Europe
...,...,...
144,Lesotho,Sub-Saharan Africa
145,Botswana,Sub-Saharan Africa
146,Rwanda,Sub-Saharan Africa
147,Zimbabwe,Sub-Saharan Africa


In [10]:
# Merge dfs and rename for sql db
allhappy_new = pd.merge(allhappy, ri_df, on="Country Name", how="left")
allhappy_new = allhappy_new.rename(columns={"Regional Indicator_y": "regional_indicator",
                                           "Country Name": "country_name",
                                           "Ladder Score": "ladder_score",
                                           "Year": "year"})
allhappy_new['ladder_score'].round(3)
allhappy_new.drop(columns="Regional Indicator_x", inplace=True)
allhappy_new

,country_name,year,ladder_score,regional_indicator
0,Afghanistan,2008,3.724,South Asia
1,Afghanistan,2009,4.402,South Asia
2,Afghanistan,2010,4.758,South Asia
3,Afghanistan,2011,3.832,South Asia
4,Afghanistan,2012,3.783,South Asia
...,...,...,...,...
2093,Lesotho,2021,3.512,Sub-Saharan Africa
2094,Botswana,2021,3.467,Sub-Saharan Africa
2095,Rwanda,2021,3.415,Sub-Saharan Africa
2096,Zimbabwe,2021,3.145,Sub-Saharan Africa


## Load: DataFrame into database

In [11]:
# Create database connection
connection_string = "postgres:Talli@0412@localhost:5432/happy_db"
engine = create_engine(f'postgresql://{connection_string}')

In [12]:
# Confirm tables
engine.table_names()

C:\Users\robin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


[]

In [13]:
allhappy_new.to_sql(name='worldhappy', con=engine, if_exists='append', index=True)

## Query to check successful load

In [14]:
# Create session for query
from sqlalchemy.orm import Session

session = Session(engine)

engine.execute("SELECT * FROM worldhappy;").fetchall()

[(0, 'Afghanistan', 2008, 3.7239999999999998, 'South Asia'),
 (1, 'Afghanistan', 2009, 4.402, 'South Asia'),
 (2, 'Afghanistan', 2010, 4.758, 'South Asia'),
 (3, 'Afghanistan', 2011, 3.832, 'South Asia'),
 (4, 'Afghanistan', 2012, 3.783, 'South Asia'),
 (5, 'Afghanistan', 2013, 3.572, 'South Asia'),
 (6, 'Afghanistan', 2014, 3.1310000000000002, 'South Asia'),
 (7, 'Afghanistan', 2015, 3.983, 'South Asia'),
 (8, 'Afghanistan', 2016, 4.22, 'South Asia'),
 (9, 'Afghanistan', 2017, 2.662, 'South Asia'),
 (10, 'Afghanistan', 2018, 2.694, 'South Asia'),
 (11, 'Afghanistan', 2019, 2.375, 'South Asia'),
 (12, 'Albania', 2007, 4.6339999999999995, 'Central and Eastern Europe'),
 (13, 'Albania', 2009, 5.485, 'Central and Eastern Europe'),
 (14, 'Albania', 2010, 5.269, 'Central and Eastern Europe'),
 (15, 'Albania', 2011, 5.867000000000001, 'Central and Eastern Europe'),
 (16, 'Albania', 2012, 5.51, 'Central and Eastern Europe'),
 (17, 'Albania', 2013, 4.551, 'Central and Eastern Europe'),
 (18, '

In [15]:
# Close session
session.close()